In [1]:
import nltk
import pandas as pd
import numpy as np


In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")    

In [87]:
df_train.head(5)

,id,keyword,location,text,target
0,1,nan,,-pron- deed reason allah forgive -pron-,1
1,4,nan,canada,forest fire near la range sask canada,1
2,5,nan,,resident ask shelter place notify officer evac...,1
3,6,nan,california,000 people receive evacuation order california,1
4,7,nan,,send photo ruby smoke pour school,1


In [67]:
# lowercasing
df_train["keyword"] = df_train["keyword"].apply(lambda x: str.lower(x) if pd.isna(x)!=True else x)
df_train["location"] = df_train["location"].apply(lambda x: str.lower(x) if pd.isna(x)!=True else x)
df_train["text"] = df_train["text"].apply(lambda x: str.lower(x) if pd.isna(x)!= True else x)
# for test data
df_test["keyword"] = df_test["keyword"].apply(lambda x: str.lower(x) if pd.isna(x)!=True else x)
df_test["location"] = df_test["location"].apply(lambda x: str.lower(x) if pd.isna(x)!=True else x)
df_test["text"] = df_test["text"].apply(lambda x: str.lower(x) if pd.isna(x)!=True else x)


In [7]:
 # remove entities
import re,string
def remove_entities(text):
    entity_prefix = ['@','#']
    for separator in string.punctuation:
        if separator not in entity_prefix:
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefix:
                words.append(word)
    return ' '.join(words)

df_train["keyword"] = df_train["keyword"].apply(lambda x: remove_entities(x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x: remove_entities(x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: remove_entities(x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: remove_entities(x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: remove_entities(x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: remove_entities(x))




In [8]:
#remove urls
df_train["keyword"] = df_train["keyword"].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x:re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\s+", ' ',x))

In [9]:
# remove puncutation
df_train["keyword"] = df_train["keyword"].apply(lambda x: re.sub(r'[^\w\s]',' ', x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x:re.sub(r'[^\w\s]',' ', x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: re.sub(r'[^\w\s]',' ', x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: re.sub(r'[^\w\s]',' ', x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: re.sub(r'[^\w\s]',' ', x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: re.sub(r'[^\w\s]',' ', x))

In [15]:
pip install symspellpy

     |████████████████████████████████| 2.6 MB 10.1 MB/s 
     |████████████████████████████████| 125 kB 52.7 MB/s 


In [27]:
dictionary_path =  "dictionary.txt"

In [28]:
# spell check
from symspellpy import SymSpell, Verbosity
sym_spell = SymSpell()

sym_spell.load_dictionary(dictionary_path,0,1)
def spelling_correction(sent):
  doc_w_correct_spelling = []
  for tok  in sent.split(" "):
    x = sym_spell.lookup(tok,Verbosity.CLOSEST,max_edit_distance=2, include_unknown = True)[0].__str__()
    y = x.split(',')[0]
    doc_w_correct_spelling.append(y)
  return " ".join(doc_w_correct_spelling)



# Spelling_Correction
df_train["keyword"] = df_train["keyword"].apply(lambda x: spelling_correction(x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x:spelling_correction(x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: spelling_correction(x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: spelling_correction(x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: spelling_correction(x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: spelling_correction(x))




In [30]:
pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 19.3 MB/s 
     |████████████████████████████████| 1.2 MB 37.4 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 880 kB 57.6 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=60fa7919416f1146bc1aaf88ba9dde2ca5eca675fb4ea200711ebcd6853e191e
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9109f0c97e17c0dd10ef9efec98f85e2dc9c5fdb8e078737c34c8cf26554891e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [35]:
# keyword extraction
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity 
import os
os.system('python -m spacy download en')
nlp = spacy.load('en')
model = spacy.load('en')


In [45]:
def extract_keywords(nlp=nlp, doc="",no_of_keywords=5,model= model):
  doc = doc.lower()
  doc = re.sub(r'(?:\@|http?://|https?\://|www)\S+',' ',doc)
  doc = re.sub(r'[^\w\s]',' ',doc)
  doc = re.sub(' \d+',' ',doc)
  doc = nlp(doc)

  pos_tag = ['VERB','NOUN','ADJ','PROPN']
  result = []
  for token in doc_:
    if (token.pos_ in pos_tag):
      result.append(token.text)
    
      
  doc_embedding = model.encode([doc])
  results_embedding = model.encode(result)

  distances = cosine_similarity(doc_embedding,result_embeddings)

  keywords = [result[index] for index in distances.argsort()[0][-no_of_keywords:]]
  return keywords




In [84]:
import warnings
warnings.filterwarnings("ignore")
for i in range(len(df_train["keyword"])):
  if pd.isnull(df_train['keyword'].iloc[i]):
    try:
      df_train['keyword'].iloc[i] = extract_keywords(nlp=nlp,doc=df_train.text.iloc[i],no_of_keywords=1,model=model)[0]
    except:
      df_train['keyword'].iloc[i] = "Nan"
  #-------------------------------------------------------------------------------------------------------------------------------------------

for i in range(len(df_test["keyword"])):
  if pd.isnull(df_test['keyword'].iloc[i]):
    try:
      df_test['keyword'].iloc[i] = extract_keywords(nlp=nlp,doc=df_test.text.iloc[i],no_of_keywords=1,model=model)[0]
    except:
      df_test['keyword'].iloc[i] = "Nan"

In [85]:
df_train

,id,keyword,location,text,target
0,1,nan,,-pron- deed reason allah forgive -pron-,1
1,4,nan,canada,forest fire near la range sask canada,1
2,5,nan,,resident ask shelter place notify officer evac...,1
3,6,nan,california,000 people receive evacuation order california,1
4,7,nan,,send photo ruby smoke pour school,1
...,...,...,...,...,...
7608,10869,nan,,giant crane hold bridge collapse nearby home h...,1
7609,10870,nan,california,harry control wild fire california northern st...,1
7610,10871,nan,,-pron- etc pm volcano hawaii help co zdtoyd8ebj,1
7611,10872,nan,portugal,police investigate bike collide car little por...,1


In [81]:
def get_location(nlp=nlp,doc=""):
  doc_ = nlp(doc)

  location = " "

  for ent in doc_.ents:
    if ent.label_ in ["GPE","ORG"]:
      location = location + ent.text + " "
  return location

for i in range(len(df_train["location"])):
  if pd.isnull(df_train['location'].iloc[i]):
    try:
      df_train['location'].iloc[i] = get_location(nlp=nlp,doc=df_train.text.iloc[i])
    except:
      df_test['location'].iloc[i] = "Nan"
 #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------     

for i in range(len(df_test["location"])):
  if pd.isnull(df_train['location'].iloc[i]):
    try:
      df_test['location'].iloc[i] = get_location(nlp=nlp,doc=df_test.text.iloc[i])
    except:
      df_test['location'].iloc[i] = "Nan"


In [83]:
df_train.head()

,id,keyword,location,text,target
0,1,nan,,-pron- deed reason allah forgive -pron-,1
1,4,nan,canada,forest fire near la range sask canada,1
2,5,nan,,resident ask shelter place notify officer evac...,1
3,6,nan,california,000 people receive evacuation order california,1
4,7,nan,,send photo ruby smoke pour school,1


In [56]:
def lemmatize(sentence):
  doc = nlp(sentence)
  lemmas = [token.lemma_ for token in doc]
  return " ".join(lemmas)

df_train["keyword"] = df_train["keyword"].apply(lambda x: lemmatize(x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x:lemmatize(x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: lemmatize(x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: lemmatize(x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: lemmatize(x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: lemmatize(x))



In [78]:
def remove_stopwords(sentence):
  doc = nlp(sentence)
  all_stopwords = nlp.Defaults.stop_words
  doc_tokens = [token.text for token in doc]
  tokens_without_sw = [word for word in doc_tokens if not word in all_stopwords]
  return " ".join(tokens_without_sw)

df_train["keyword"] = df_train["keyword"].apply(lambda x: remove_stopwords(x) if pd.isna(x) !=True else x)
df_train["location"] = df_train["location"].apply(lambda x:remove_stopwords(x) if pd.isna(x) != True else x)
df_train["text"] = df_train["text"].apply(lambda x: remove_stopwords(x))

df_test["keyword"] = df_test["keyword"].apply(lambda x: remove_stopwords(x) if pd.isna(x) !=True else x)
df_test["location"] = df_test["location"].apply(lambda x: remove_stopwords(x) if pd.isna(x) !=True else x)
df_test["text"] = df_test["text"].apply(lambda x: remove_stopwords(x))

In [79]:
df_train

,id,keyword,location,text,target
0,1,nan,,-pron- deed reason allah forgive -pron-,1
1,4,nan,canada,forest fire near la range sask canada,1
2,5,nan,,resident ask shelter place notify officer evac...,1
3,6,nan,california,000 people receive evacuation order california,1
4,7,nan,,send photo ruby smoke pour school,1
...,...,...,...,...,...
7608,10869,nan,,giant crane hold bridge collapse nearby home h...,1
7609,10870,nan,california,harry control wild fire california northern st...,1
7610,10871,nan,,-pron- etc pm volcano hawaii help co zdtoyd8ebj,1
7611,10872,nan,portugal,police investigate bike collide car little por...,1


In [60]:
#model
import tensorflow_hub as hub

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
embed = hub.load(model)

In [61]:
from sklearn.utils import shuffle

train = shuffle(df_train,random_state = 42).reset_index(drop=True) #shuffle the train data
y=np.array(df_train["target"].tolist()) #convert the target column into a numpy array

key_embed = embed(df_train.keyword.tolist()) #keyword embeddings
loc_embed = embed(df_train.location.to_list())#location embeddings
text_embed = embed(train.text.to_list())#text embeddings

In [62]:
from tensorflow.keras.layers import Input , Dense , Flatten , concatenate , Dropout
from tensorflow.keras import Model

keyword_input = Input(shape=(key_embed.shape[1],))
location_input = Input(shape=(loc_embed.shape[1],))
